# Dataset download and save in drive

In [ ]:
!wget http://images.cocodataset.org/zips/val2014.zip --verbose

In [ ]:
!unzip val2014.zip -d /content/CocoVal2014

In [27]:
!rm /content/val2014.zip

# Dataset prepratation

In [28]:
DATASET_PATH = '/content/CocoVal2014'

In [29]:
import os

FINAL_PATH = '/content/CocoVal2014-modified'

if not os.path.exists(FINAL_PATH):
  os.makedirs(FINAL_PATH)

In [30]:
from PIL import Image,ImageChops

def is_greyscale(im):
    if im.mode not in ("L", "RGB"):
        raise ValueError("Unsuported image mode")

    if im.mode == "RGB":
        rgb = im.split()
        if ImageChops.difference(rgb[0],rgb[1]).getextrema()[1]!=0: 
            return False
        if ImageChops.difference(rgb[0],rgb[2]).getextrema()[1]!=0: 
            return False
    return True

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

for subdir, dirs, files in os.walk(DATASET_PATH):
    for file in tqdm(files):
        filepath = subdir + os.sep + file
        image = Image.open(filepath)

        if is_greyscale(image):
          continue

        new_image = image.resize((64, 64))
        new_image.save(FINAL_PATH + os.sep + file)

In [32]:
!rm -rf /content/CocoVal2014/

# Model

In [33]:
FINAL_PATH = '/content/CocoVal2014-modified'

In [ ]:
!nvidia-smi

In [35]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F
from torchvision.utils import save_image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [37]:
class depthwise_separable_conv(nn.Module):
    def __init__(self, nin, kernels_per_layer, nout):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin * kernels_per_layer, kernel_size=5, padding=2, groups=nin)
        self.pointwise = nn.Conv2d(nin * kernels_per_layer, nout, kernel_size=1)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

In [38]:
def swish(x):
    return x * torch.sigmoid(x)

In [39]:
class ChannelSELayer(nn.Module):
    def __init__(self, num_channels, reduction_ratio=2):
        super(ChannelSELayer, self).__init__()
        num_channels_reduced = num_channels // reduction_ratio
        self.reduction_ratio = reduction_ratio
        self.fc1 = nn.Linear(num_channels, num_channels_reduced, bias=True)
        self.fc2 = nn.Linear(num_channels_reduced, num_channels, bias=True)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor):
        batch_size, num_channels, H, W = input_tensor.size()
        squeeze_tensor = input_tensor.view(batch_size, num_channels, -1).mean(dim=2)

        fc_out_1 = self.relu(self.fc1(squeeze_tensor))
        fc_out_2 = self.sigmoid(self.fc2(fc_out_1))

        a, b = squeeze_tensor.size()
        output_tensor = torch.mul(input_tensor, fc_out_2.view(a, b, 1, 1))
        return output_tensor

In [40]:
class dec_res(nn.Module):
  def __init__(self,in_channel):
    super(dec_res,self).__init__()
    
    self.bn1 = nn.BatchNorm2d(in_channel)
    
    self.c1 = nn.Conv2d(in_channels=in_channel,out_channels=2*in_channel,kernel_size=1,stride=1,padding=0)
    self.bn2 = nn.BatchNorm2d(2*in_channel)

    self.c2 = nn.Conv2d(in_channels=2*in_channel,out_channels=4*in_channel,kernel_size=1,stride=1,padding=0)
    self.bn3 = nn.BatchNorm2d(4*in_channel)

    self.c3 = nn.Conv2d(in_channels=4*in_channel,out_channels=8*in_channel,kernel_size=1,stride=1,padding=0)
    self.bn4 = nn.BatchNorm2d(8*in_channel)

    self.dc1 = depthwise_separable_conv(nin=8*in_channel,kernels_per_layer=3,nout=8*in_channel)

    self.bn5 = nn.BatchNorm2d(8*in_channel)
    self.c4 = nn.Conv2d(in_channels=8*in_channel,out_channels=4*in_channel,kernel_size=1,stride=1,padding=0)

    self.bn6 = nn.BatchNorm2d(4*in_channel)
    self.c5 = nn.Conv2d(in_channels=4*in_channel,out_channels=2*in_channel,kernel_size=1,stride=1,padding=0)

    self.bn7 = nn.BatchNorm2d(2*in_channel)
    self.c6 = nn.Conv2d(in_channels=2*in_channel,out_channels=in_channel,kernel_size=1,stride=1,padding=0)

    self.bn8 = nn.BatchNorm2d(in_channel)

    self.SE = ChannelSELayer(in_channel)

  def forward(self,x1):
    x = self.c1(self.bn1(x1))
    
    x = swish(self.bn2(x))
    x = self.c2(x)
    
    x = swish(self.bn3(x))
    x = self.c3(x)

    x = swish(self.bn4(x))
    x = self.dc1(x)

    x = swish(self.bn5(x))
    x = self.c4(x)

    x = swish(self.bn6(x))
    x = self.c5(x)

    x = swish(self.bn7(x))
    x = self.c6(x)
    
    x = self.bn8(x)
    x = self.SE(x)
    return x+x1

class enc_res(nn.Module):
  def __init__(self,in_channel):
    super(enc_res,self).__init__()

    self.bn1 = nn.BatchNorm2d(in_channel)

    self.c1 = nn.Conv2d(in_channels=in_channel,out_channels=2*in_channel,kernel_size=3,stride=1,padding=1)
    self.bn2 = nn.BatchNorm2d(2*in_channel)

    self.c2 = nn.Conv2d(in_channels=2*in_channel,out_channels=4*in_channel,kernel_size=3,stride=1,padding=1)
    self.bn3 = nn.BatchNorm2d(4*in_channel)

    self.c3 = nn.Conv2d(in_channels=4*in_channel,out_channels=8*in_channel,kernel_size=3,stride=1,padding=1)
    self.bn4 = nn.BatchNorm2d(8*in_channel)

    self.c4 = nn.Conv2d(in_channels=8*in_channel,out_channels=4*in_channel,kernel_size=3,stride=1,padding=1)
    self.bn5 = nn.BatchNorm2d(4*in_channel)

    self.c5 = nn.Conv2d(in_channels=4*in_channel,out_channels=2*in_channel,kernel_size=3,stride=1,padding=1)
    self.bn6 = nn.BatchNorm2d(2*in_channel)

    self.c6 = nn.Conv2d(in_channels=2*in_channel,out_channels=in_channel,kernel_size=3,stride=1,padding=1)
    self.bn7 = nn.BatchNorm2d(in_channel)

    self.SE = ChannelSELayer(in_channel)
  def forward(self,x1):
    x = swish(self.bn1(x1))
    x = self.c1(x)

    x = swish(self.bn2(x))
    x = self.c2(x)

    x = swish(self.bn3(x))
    x = self.c3(x)

    x = swish(self.bn4(x))
    x = self.c4(x)

    x = swish(self.bn5(x))
    x = self.c5(x)

    x = swish(self.bn6(x))
    x = self.c6(x)

    x = swish(self.bn7(x))
    x = self.SE(x)
    return x+x1

In [41]:
class NVAE(nn.Module):
  def __init__(self,start_channel,original_dim):
    super(NVAE,self).__init__()
    self.original_dim = original_dim
    self.conv1 = nn.Conv2d(in_channels=start_channel,out_channels=8,kernel_size=3,stride=1,padding=1)
    self.encblock1 = enc_res(8)
    self.dsconv1 = nn.Conv2d(in_channels=8,out_channels=8,kernel_size=2,stride=2,padding=0)
    self.encblock2 = enc_res(8)
    self.dsconv2 = nn.Conv2d(in_channels=8,out_channels=8,kernel_size=2,stride=2,padding=0)

    self.qmu1 = nn.Linear(original_dim*original_dim*2,original_dim*original_dim*2)
    self.qvar1 = nn.Linear(original_dim*original_dim*2,original_dim*original_dim*2)
    
    self.qmu0 = nn.Linear(original_dim*original_dim//2,original_dim*original_dim//2)
    self.qvar0 = nn.Linear(original_dim*original_dim//2,original_dim*original_dim//2)

    self.pmu1 = nn.Linear(original_dim*original_dim*2,original_dim*original_dim*2)
    self.pvar1 = nn.Linear(original_dim*original_dim*2,original_dim*original_dim*2)

    self.decblock1 = dec_res(8)
    self.usconv1 = nn.ConvTranspose2d(in_channels=8,out_channels=8,kernel_size=2,stride=2,padding=0)
    self.decblock2 = dec_res(16)
    self.usconv2 = nn.ConvTranspose2d(in_channels=16,out_channels=16,kernel_size=2,stride=2,padding=0)
    self.decblock3 = dec_res(16)
    self.finconv = nn.Conv2d(in_channels=16,out_channels=start_channel,kernel_size=3,stride=1,padding=1)
  
  def forward(self,x):
    z1 = self.dsconv1(self.encblock1(self.conv1(x)))
    z0 = self.dsconv2(self.encblock2(z1))

    qmu0 = self.qmu0(z0.reshape(z0.shape[0],self.original_dim*self.original_dim//2))
    qvar0 = self.qvar0(z0.reshape(z0.shape[0],self.original_dim*self.original_dim//2))

    qmu1 = self.qmu1(z1.reshape(z1.shape[0],self.original_dim*self.original_dim*2))
    qvar1 = self.qvar1(z1.reshape(z1.shape[0],self.original_dim*self.original_dim*2))

    stdvar0 = qvar0.mul(0.5).exp_()
    stdvar1 = qvar1.mul(0.5).exp_()

    e0 = torch.randn(qmu0.shape).to(device)
    ez0 = qmu0+e0*stdvar0
    ez0 = ez0.reshape(ez0.shape[0],8,self.original_dim//4,self.original_dim//4)
    ez1 = self.usconv1(self.decblock1(ez0))

    pmu1 = self.pmu1(ez1.reshape(ez1.shape[0],self.original_dim*self.original_dim*2))
    pvar1 = self.pvar1(ez1.reshape(ez1.shape[0],self.original_dim*self.original_dim*2))

    pstdvar1 = pvar1.mul(0.5).exp_()

    e2 = torch.randn(qmu1.shape).to(device)
    ez2 = pmu1+qmu1 + e2*pstdvar1*stdvar1
    ez2 = ez2.reshape(ez2.shape[0],8,self.original_dim//2,self.original_dim//2)
    
    final = torch.cat((ez1,ez2),1)

    recons = nn.Sigmoid()(self.finconv(self.decblock3(self.usconv2(self.decblock2(final)))))

    return qmu0,qvar0,qmu1,qvar1,pmu1,pvar1,recons

  def sample(self,bs):
    e = torch.randn([bs,8,self.original_dim//4,self.original_dim//4]).to(device)
    ez1 = self.usconv1(self.decblock1(e))

    pmu1 = self.pmu1(ez1.reshape(ez1.shape[0],self.original_dim*self.original_dim*2))
    pvar1 = self.pvar1(ez1.reshape(ez1.shape[0],self.original_dim*self.original_dim*2))

    stdvar1 = pvar1.mul(0.5).exp_()

    e1 = torch.randn([ez1.shape[0],self.original_dim*self.original_dim*2]).to(device)
    e1 = pmu1 + e1*stdvar1
    e1 = e1.reshape(e1.shape[0],8,self.original_dim//2,self.original_dim//2)
    recons = nn.Sigmoid()(self.finconv(self.decblock3(self.usconv2(self.decblock2(torch.cat((ez1,e1),1))))))

    return recons

  def loss(self,x):
    qmu0,qvar0,qmu1,qvar1,pmu1,pvar1,recons = self.forward(x)
    klz0 = 0.5*torch.sum(torch.square(qmu0)+qvar0.exp()-qvar0-1)/x.shape[0]
    klz1 = 0.5*torch.sum(torch.square(qmu1)/pvar1.exp()+qvar1.exp()-qvar1-1)
    reconsloss = nn.BCELoss()(recons,x)
    return klz0,klz1,reconsloss

In [42]:
batch_size=64

In [43]:
transform = transforms.Compose([transforms.ToTensor()])

In [44]:
from os import listdir
from PIL import Image

class CocoDataloader(object):
    def __init__(self, data_dir, transform=None):
        self.transform = transform
        self.image_names = [os.path.join(data_dir, img) for img in listdir(data_dir) if os.path.join(data_dir, img)]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):

        image = Image.open(self.image_names[idx])

        if self.transform:
            image = self.transform(image)

        return image, self.image_names[idx]

In [45]:
from torch.utils.data import DataLoader

train = CocoDataloader(FINAL_PATH, transform=transforms.Compose([transforms.ToTensor()]))
loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)

In [46]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [47]:
model = NVAE(3,64).to(device)
optim = torch.optim.Adamax(model.parameters())

In [48]:
epochs = 20

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.image as mpimg

for epoch in range(epochs):
    minloss = 1
    running_kl0_loss=0
    running_recons_loss=0
    running_kl1_loss=0
    num_images=0
    for i, (img, image_name) in enumerate(loader):
      img = img.to(device)
      optim.zero_grad()
      klz0,klz1,recons = model.loss(img)
      loss=recons+epoch*0.001*klz0+epoch*0.001*klz1
      loss.backward()
      optim.step()
      running_kl0_loss = running_kl0_loss + klz0.item()*len(img)
      running_kl1_loss = running_kl1_loss + klz1.item()*len(img)
      running_recons_loss = running_recons_loss + recons.item()*len(img)

      num_images= num_images+len(img)
    print('epoch: '+str(epoch)+' kl0_loss: '+str(running_kl0_loss/num_images)+' recons_loss: '+str(running_recons_loss/num_images)+' kl1_loss: '+str(running_kl1_loss/num_images))
    img = model.sample(1).cpu().detach().reshape(64, 64, 3).numpy()
    plt.imshow(img)
    plt.savefig(str(epoch)+".png")
    plt.show()